In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from linopy import Model

# Technologies data:
tech_param = pd.read_csv('Data/technologies.csv',index_col=0)
# Storages data
sto_param = pd.read_csv("Data/storage_technologies.csv", index_col=0).to_xarray()
sto_param_cold = pd.read_csv("Data/storage_cold_technologies.csv", index_col=0).to_xarray()
# Demand:
demand = pd.read_csv('Data/demand_hourly_Abu_Dhabi.csv',index_col=0, delimiter = ';').to_xarray()
# COP
cop = pd.read_csv('Data/COP.csv',index_col=0, delimiter= ',')
# Global parameters:
general_parameters = pd.read_csv('Data/global_parameters.csv',index_col=0, delimiter= ',')
# Airchiller data:
conv_tech_param = pd.read_csv('Data/conversion_technologies.csv', index_col =0, delimiter = ',').to_xarray()

# Renewable generation:
AF = pd.read_csv('Data/AF.csv',index_col=0, delimiter = ';') # AF is given in the file only for the renewables, we need to add columns for the other technologies.
for tech in tech_param.index:
    if tech not in AF.columns:
        AF.loc[:,tech] = tech_param.loc[tech, "Availability"] 
tech_param = tech_param.to_xarray()

In [2]:
### Sets
Time = demand.get_index("Time")
Tech = tech_param.get_index("Technologies")

# Technologies related
TECH_MAX_CAP = tech_param['Maximum capacity']

MARGINAL_FUEL_COST = tech_param['Fuel costs']/tech_param['Rated efficiency']
CO2_INTENSITY = tech_param['Fuel CO2 content']/tech_param['Rated efficiency']
MARGINAL_CO2_EMISSION = tech_param['Fuel CO2 content']
TECH_PHI = tech_param["Discount rate"]/(1-(1+tech_param["Discount rate"])**(-tech_param["Lifetime"]))
TECH_IC = TECH_PHI*tech_param['Investment cost']
DELTA_MAX_UP = tech_param['Ramp-up rate']
DELTA_MAX_DOWN = tech_param['Ramp-down rate']
AF = AF[Tech]
AF  = xr.DataArray(AF,coords=[Time,Tech])
LEGACY = tech_param["Legacy capacity"]

# Electrical Storage related
STO_MAX_CAP = sto_param["Maximum capacity"]
STO_PHI = sto_param["Discount rate"]/(1-(1+sto_param["Discount rate"])**(-sto_param["Lifetime"]))
STO_IC = STO_PHI*sto_param["Investment cost"]
EFF_D = sto_param["Discharge efficiency"]
EFF_C = sto_param["Charge efficiency"]
SOC_MAX = sto_param["Maximum state of charge"]
SOC_MIN = sto_param["Minimum state of charge"]
STO_dT = sto_param["Storage duration"]

# Electrical Storage related
STO_MAX_CAP_COLD = sto_param_cold["Maximum capacity"]
STO_PHI_COLD = sto_param_cold["Discount rate"]/(1-(1+sto_param_cold["Discount rate"])**(-sto_param_cold["Lifetime"]))
STO_IC_COLD = STO_PHI_COLD*sto_param_cold["Investment cost"]
EFF_D_COLD = sto_param_cold["Discharge efficiency"]
EFF_C_COLD = sto_param_cold["Charge efficiency"]
SOC_MAX_COLD = sto_param_cold["Maximum state of charge"]
SOC_MIN_COLD = sto_param_cold["Minimum state of charge"]
STO_dT_COLD = sto_param_cold["Storage duration"]


# Links Related
conv_tech_phi = conv_tech_param['Discount rate']/(1-(1+conv_tech_param['Discount rate'])**(-conv_tech_param['Lifetime']))
CONV_TECH_IC = conv_tech_phi*conv_tech_param['Investment cost']
CONV_TECH_MAX_CAP = conv_tech_param['Maximum capacity']

COP = cop["COP"]
COP  = xr.DataArray(COP,coords=[Time])
DEMAND_elec = demand["Campus_elec"]
DEMAND_cold = demand["Campus_cold"]

# Configuration and scenario parameters
TS = float(general_parameters.Value.loc['Time step'])
VOLL = float(general_parameters.Value.loc['VOLL'])
CO2_PRICE = float(general_parameters.Value.loc['CO2_price'])
CO2_BOUND = float(general_parameters.Value.loc['CO2_bound'])

In [5]:
### Create the model instance

m = Model()

### Create the variables

cap_tech = m.add_variables(lower = 0, coords = [Tech], name = 'cap_tech') # Installed capacity of the technologies [MW]
g = m.add_variables(lower = 0, coords = [Tech, Time], name = 'g')         # Generated power [MW]
fcd = m.add_variables(lower = 0, coords = [Tech, Time], name = 'fcd')     # Fuel cost definition [EUR/h]
ccd = m.add_variables(lower = 0, coords= [Tech, Time], name = 'ccd')      # CO2 cost definition [EUR/h]
cap_sto = m.add_variables(lower = 0, name = 'cap_sto')                    # Installed capacity of the storages [MWh]
sd = m.add_variables(lower = 0, coords = [Time], name = 'sd')             # Storage discharge [MW]
sc = m.add_variables(lower = 0, coords = [Time], name = 'sc')             # Storage charge [MW]
se = m.add_variables(lower = 0, coords = [Time], name = 'se')             # Stored energy [MWh]


inj = m.add_variables(lower = 0, coords= [Time], name = 'inj') 
ens_elec = m.add_variables(lower = 0, coords = [Time], name = 'ens')      # Energy no served [MWh]
ens_cold = m.add_variables(lower = 0, coords = [Time], name = 'ens_cold')

cap_conv_tech = m.add_variables(lower = 0, name = "cap_conv_tech") 
#NO curtailment considered here

#Cold battery variables
sc_cold = m.add_variables(lower = 0, coords= [Time], name = 'sc_cold')     # Storage charge (Ice formation) [MW_cold]
se_cold = m.add_variables(lower = 0, coords= [Time], name = 'se_cold')     # Stored energy [MWh_cold]

cap_sto_cold = m.add_variables(lower = 0, name = 'cap_sto_cold')           # Installed capacity of the storages [MWh_cold] 



### Create the constraints

# Technology sizing
cap_max_tech = m.add_constraints(cap_tech <= TECH_MAX_CAP, name = "cap_max_tech")
power_max = m.add_constraints(g <= AF*cap_tech, name= 'power_max')

# Ramping up/down constraints
power_rampup_initial = m.add_constraints(g.loc[:,:Time[0]] - g.loc[:,Time[len(Time)-1]:] <= DELTA_MAX_UP*cap_tech , name = 'power_rampup_initial')
power_rampup = m.add_constraints(g.loc[:, Time[1:]] <= g.shift(Time = 1) + DELTA_MAX_UP*cap_tech, name = 'power_rampup')

power_rampdown_initial = m.add_constraints(g.loc[:,:Time[0]] - g.loc[:,Time[len(Time)-1]:]  >= - DELTA_MAX_DOWN*cap_tech, name = 'power_rampdown_initial')
power_rampdown = m.add_constraints(g.loc[:, Time[1:]]>= g.shift(Time = 1) - DELTA_MAX_DOWN*cap_tech, name = 'power_rampdown')

# Electrical Storage sizing + cyclic boundary condition
cap_max_sto = m.add_constraints(cap_sto <= STO_MAX_CAP, name = "cap_max_sto")
storage_balance = m.add_constraints(se.loc[Time[1:]] - se.shift(Time = 1) == TS*EFF_C*sc.loc[Time[1:]] -TS*sd.loc[Time[1:]]/EFF_D, name = 'storage_balance')
storage_charge_max = m.add_constraints(sc <= cap_sto/STO_dT, name = 'storage_charge_max')
storage_in_fin = m.add_constraints(se.loc[:Time[0]] == se.loc[Time[len(Time)-1]:] , name = 'storage_in_fin')

storage_discharge_max = m.add_constraints(sd <= cap_sto/STO_dT, name = 'storage_discharge_max')
storage_energy_max = m.add_constraints(se <= SOC_MAX*cap_sto, name = 'storage_energy_max' )
storage_energy_min = m.add_constraints(se >= SOC_MIN*cap_sto, name = 'storage_energy_min' )


#Cold storage sizing + cyclic boundary condition
cap_max_sto_cold = m.add_constraints(cap_sto_cold <= STO_MAX_CAP_COLD, name = "cap_max_sto_cold")
storage_in_fin_cold = m.add_constraints(se_cold.loc[:Time[0]] == se_cold.loc[Time[len(Time)-1]:] , name = 'storage_in_fin_cold')
storage_initial_state_cold = m.add_constraints(se_cold.loc[:Time[0]] - se_cold.loc[Time[len(Time)-1]:] == TS*EFF_C_COLD*sc_cold.loc[Time[len(Time)-1]:] - TS*DEMAND_cold.loc[Time[len(Time)-1]:]/EFF_D_COLD, name = 'storage_initial_state_cold')
storage_balance_cold = m.add_constraints(se_cold.loc[Time[1:]] - se_cold.shift(Time = 1) == TS*EFF_C_COLD*sc_cold.loc[Time[1:]] -TS*DEMAND_cold.loc[Time[1:]]/EFF_D_COLD, name = 'storage_balance_cold')

storage_charge_max_cold = m.add_constraints(sc_cold <= cap_sto_cold/STO_dT_COLD, name = 'storage_charge_max_cold')
storage_energy_max_cold = m.add_constraints(se_cold <= SOC_MAX_COLD*cap_sto_cold, name = 'storage_energy_max_cold' )
storage_energy_min_cold = m.add_constraints(se_cold >= SOC_MIN_COLD*cap_sto_cold, name = 'storage_energy_min_cold' )

# Market clearing
mcc = m.add_constraints(g.sum(dims = 'Technologies') + ens_elec + sd - sc - inj   ==  DEMAND_elec , name = 'mcc')
mcc2 = m.add_constraints(inj*COP + ens_cold == sc_cold, name = 'mcc2')

# Limit on energy not served
ens_lim = m.add_constraints(ens_elec<=DEMAND_elec, name = 'ens_lim')
ens_lim2 = m.add_constraints(ens_cold<=DEMAND_cold, name = 'ens_lim2')
links_max = m.add_constraints(inj*COP <= cap_conv_tech, name="max_flow_conv_tech")

# Costs
fuel_cost = m.add_constraints(fcd == MARGINAL_FUEL_COST*g, name="fuel_cost")

#There are no CO2 costs


# the objective function
total_cost = (TECH_IC*cap_tech).sum(dim="Technologies") + (CONV_TECH_IC*cap_conv_tech) + (STO_IC*cap_sto) + (STO_IC_COLD*cap_sto_cold) + (TS*fcd + TS*ccd).sum(dims = ['Technologies', 'Time']) + (VOLL*TS*ens_elec).sum(dims ='Time') + (VOLL*TS*ens_cold).sum(dims ='Time')
m.add_objective(total_cost)

In [6]:
m.solve("gurobi", BarHomogeneous = True, BarConvTol=1e-4)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2551043


GurobiError: SSL: no alternative certificate subject name matches target hostname 'token.gurobi.com' (code 60, command POST https://token.gurobi.com/api/v1/tokens)

In [ ]:
print('capacité maximum de la baterie:', cap_sto.solution.values)
print('capacité maximum du stockage de froid:', cap_sto_cold.solution.values)
print('Valeur maximum de ma charge durant l année:', sc_cold.solution.max(dim='Time').values)
print('capacité maximum du stockage de froid:', sc_cold.solution.values[:20]) 
print('capacité maximum du stockage de froid:', sc_cold.solution.sum().values) 
print('capacité maximum du stockage de froid:', DEMAND_cold.sum().values) 